In [ ]:
#Note: Some of the functionalities in this notebook do not work in the current PyHAT version.

## An example that demonstrates the manipulation, visualization, and analysis of M-cubed (M3) data

In [ ]:
#The following two lines may be required to point the notebook to your PyHAT install
import sys
sys.path.append('[PATH TO PYHAT REPO]')

import libpyhat as phat
import libpyhat.pyhat_io.io_moon_mineralogy_mapper as io_moon_mineralogy_mapper
%pylab inline

## Basic M3 I/O

In [ ]:
m = io_moon_mineralogy_mapper.open(phat.examples.get_path('M3_4030seg_L2_aristcrater.tif'))

## Visualize one of the bands by index

In [ ]:
imshow(m[0], cmap='Greys')
colorbar()

## Get the available wavelengths

In [ ]:
#For some reason, this file does not have the wavelength attribute which breaks everything that follows.
m.wavelengths

## Visualize by wavelength with tolerances enabled

The idea behind tolerances is to be able to control how many decimal places are included for label based access.  The default tolerance is 2, meaning that all wavelength labels (not the actual values, but the labels by which data are accessible) is rounded to two decimal places.

Below, we access the private variable `_original_wavelengths` to demonstrate that the actual wavelengths are being preserverd.  We can compate the final entry `700.539978` with the label representation `700.54` (above).

In [ ]:
m._original_wavelengths[:5]

It is possible to alter the labels by altering the tolerance value.  For example, below the tolerance is set to five decimal places.

In [ ]:
m.tolerance = 5
m.wavelengths[:5]  # Print the first 5 wavelengths

In [ ]:
m.tolerance = 2

## Label Based Access

In [ ]:
imshow(m.loc[700], cmap='Greys')
colorbar()

In [ ]:
m.wavelengths

In [ ]:
imshow(m.loc[2616], cmap='Greys')
colorbar()

## Label Based Access in an ROI
Suppose that the crater is of particular interest.  In this isntance, a slice can be taken from the data where the indexer looks like `[1149, 20:40, 0:20]`.  The first element (`1149`) is the label wavelength.  The second (`20:40`, y-axis) and third (`0:20`, x-axis) are positional in pixel space and are in `start:stop` form.

In [ ]:
imshow(m.loc[1149, 20:40, 0:20], cmap='Blues')
colorbar()

## Label Based Access for an RGB False Color
Here 3 bands are pulled from the image and not ROI is defined.  Notice that the three bands are specified inside of a list `[1978, 540, 770]` and that that list is inside of the `loc` list, e.g., `[[1978, 540, 770]]`.  If one wanted to also specify an ROI, the syntax would be `[[1978, 540, 770], ystart:ystop, xstart:xstop]`.

In [ ]:
arr = m.loc[[1978, 540, 770]]
arr = np.moveaxis(arr, 0, -1)  # The axis movement is required for a 3 band plot.
imshow(arr)

In [ ]:
from libpyhat.derived.m3 import pipe, new, development_funcs, supplemental

## What are the available derived products for M3.pipe?

In [ ]:
available = [i for i in dir(pipe) if not i.startswith('__')]
print(available)

### Band Depth @ 620

In [ ]:
imshow(pipe.bd1900(m), cmap='Greens')
colorbar()

### H20 1st Algorithm

In [ ]:
# TODO: Does this still exist? Find out to see if we can delete this cell.

imshow(development_funcs.h2o1(m), cmap='Blues')
colorbar()

In [ ]:
imshow(pipe.twoum_slope(m), cmap='Reds')
colorbar()

## Derived Product with an ROI
This is currently not an ideal solution and we are thinking about how better to implement this. In short, the convenience functions, e.g., `pipe.twoum_slope` expect to get a `GeoDataset` object (what the m3 object is in this notebook).  When we start to slice (subset to an ROI) a standard numpy array is returned.  Therefore, the `pipe.twoum_slope` function will not work directly on a subset. The `pipe.twoum_slope` dispatches to another function that does the heavy lifting.  In the example, below, we call that function directly.  

In the short term, we suggest that end users subset their images to some ROI using a tool like gdal_translate and use those data within PyHAT.

In [ ]:
from libpyhat.derived.m3 import pipe_funcs
imshow(pipe_funcs.twoum_slope_func(m.loc[[1578, 2538], 20:40, 0:20]), cmap='Oranges')
colorbar()